# Análise de dados utilizando SQLite

## 1. Introdução

* O objetivo desse projeto é analisar uma base de dados relacional de uma loja de música virtual (similar ao i-tunes da apple na época em que era comum a compra de músicas em .mp3).


* Nossa base de dados é uma base em **SQLite**, organizada conforme o diagrama abaixo. As chaves primárias de cada tabela estão destacas mais escuro, e suas relações com as chaves estrangeiras estão indicadas pelas linhas. De um modo geral a base respeita as 3 principais regras normais de garantia de integridade dos dados e performace do banco.


<img src="https://i.imgur.com/2UgQ29p.png" />


* A partir da análise do banco, vamos responder as seguinte perguntas:

  
 > 1. Qual o gênero musical que vendeu mais músicas nos EUA?
 > 2. Quais funcionários da loja venderam mais? Qual o perfil desses funcionários?
 > 3. Como foram as vendas por país? Quantos clientes existem em cada país? Qual o valor médio gasto por cliente em cada país? etc...
 > 4. Os clientes tem comprado mais músicas avulsas ou álbuns inteiros?


* Esse projeto foi desenvolvido por mim como parte do curso de **"Intermediate SQL for data Analysis" do Dataquest**, um site americano que oferece cursos pagos de ciência de dados. O Dataquest oferece **projetos guiados** ao término de cada módulo, onde os alunos aplicam os conceitos aprendidos no módulo. Nos projetos guiados são utilizados datasets reais e os alunos recebem instruções gerais de como resolver o problema proposto. O site oferece a "solução" para cada projeto guiado, que pode ser encontrada no seguinte link https://github.com/dataquestio/solutions/blob/master/Mission191Solutions.ipynb
Como estou iniciando em data-science, obviamente tive que recorrer à solução disponibilizada pelo site, no entanto, **escrevi meu próprio código e em várias etapas melhorei o código em relação à solução apresentada**.




## 2. Conectando à base

* Nesse projeto vamos fazer nossas queries direto do jupyter notebook nós conectando ao nosso banco, cujo nome é 'chinook.db'


* Para isso vamos utilizar o comando (magic) do jupyter notebook **%load_ext sql**, do ipython, que carrega o módulo SQL e nos permite conectar à base.


* Para conectar à nossa bse do sqlite vamos precisar utilizar passar o comando a seguir, com a string de conexão à base apenas uma vez **%sqlite:///chinook.db**


* Para escrever nossas queries, basta colocar **%%sql** no começo das células, sem a necessidade de criar 

In [1]:
%%capture
!pip install ipython-sql

In [2]:
%load_ext sql
%sql sqlite:///chinook.db

In [3]:
%%sql
Select name, type
from sqlite_master
where type in ('table','view');

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## 3. Respondendo às perguntas sobre a loja de músicas

## 3.1. Qual o gênero musical que vendeu mais músicas nos EUA?

* Para responder a esta pergunta eu fiz eu fiz 2 subqueries 1 uma query principal


* As subqueries aparecem antes da query principal, utilizando a função **with**


* **Subquery "usa_tracks"** 
> * Contém 3 informações principais: O país, o gênero, e a contagem de músicas vendidas para determinado gênero. 
> * Para isso, fiz vários inner joins utilizando as chaves principais de cada tabela. 
> * Além disso utilizei o comando group by para agrupar por gênero e por país.  
> * Filtrei apenas o país USA
> * Ordenei por número de músicas vendidas
* **Subquery total_sold_usa**
> * Como pretendemos mostrar o resultado também por porcentagem, a função dessa subquery é calcular o total de músicas vendidas nos EUA
* **Query principal** divide o valor absoluto por gênero (primeira subquerry) pelo total de tracks nos EUA (segunda subquery)

In [4]:
%%sql

with usa_tracks as
   (select
       c.country,
       g.name as genre,
       count(il.invoice_line_id) as absolute_sales
      from customer c
      inner join invoice i on c.customer_id = i.customer_id
      inner join invoice_line il on i.invoice_id = il.invoice_id
      inner join track t on il.track_id = t.track_id
      inner join genre g on t.genre_id = g.genre_id
      group by g.name, c.country
      having c.country = "USA"
      order by absolute_sales desc
     ),

total_sold_usa as
 (select 
    country, 
    sum(absolute_sales) as total 
  from usa_tracks)

select 
    usa_tracks.genre, 
    usa_tracks.absolute_sales,
    round(cast(usa_tracks.absolute_sales as float) / cast(total_sold_usa.total as float), 4) as percentage
    from usa_tracks
    inner join total_sold_usa on usa_tracks.country = total_sold_usa.country
    

 * sqlite:///chinook.db
Done.


genre,absolute_sales,percentage
Rock,561,0.5338
Alternative & Punk,130,0.1237
Metal,124,0.118
R&B/Soul,53,0.0504
Blues,36,0.0343
Alternative,35,0.0333
Latin,22,0.0209
Pop,22,0.0209
Hip Hop/Rap,20,0.019
Jazz,14,0.0133


Vemos que **rock, punk & alternative e metal** (que também são tipos de rock) correspondem a **mais de 70%** das músicas vendidas.

## 3.2. Quais funcionários venderam mais? Qual o perfil desses funcionários?

* Para responder esta pergunta, precisamos seguir o caminho desde os recibos (tabela invoice), passando pelo cliente (tabela customer) até a tabela com dados dos funcionários (tabela employee). Para isso fizemos inner joins entre essas tabelas.
* Concatenamos o primeiro e o segundo nome dos funcionários usando || " " ||

In [5]:
%%sql

select
    e.first_name || " " || e.last_name as employee_name,
    e.title,
    e.reports_to,
    e.hire_date,
    sum(i.total) total_sales
 from employee e
 left join customer c on e.employee_id = c.support_rep_id
 inner join invoice i on c.customer_id = i.customer_id
 group by e.employee_id
 order by total_sales desc

 * sqlite:///chinook.db
Done.


employee_name,title,reports_to,hire_date,total_sales
Jane Peacock,Sales Support Agent,2,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,Sales Support Agent,2,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,Sales Support Agent,2,2017-10-17 00:00:00,1393.920000000002


A funcionária que mais vendeu foi **Jane Peacock**, que é a **funcionária mais antiga** dentre os resultados apresentados.

Nossa lista retornou apenas 3 nomes, o que a princípio parece estranho. Fazendo a query abaixo podemos confirmar que os outros funcionários não trabalham com vendas.

In [6]:
%%sql
select first_name, title from employee

 * sqlite:///chinook.db
Done.


first_name,title
Andrew,General Manager
Nancy,Sales Manager
Jane,Sales Support Agent
Margaret,Sales Support Agent
Steve,Sales Support Agent
Michael,IT Manager
Robert,IT Staff
Laura,IT Staff


## 3.3. Analisando as vendas por país e a o valor médio gasto por comprador por país

* Eu criei 2 sub querries antes da query principal utilizando **with**
* Subquerry 1 **customer_agregate**
> * Essa sub querry é um agrupamento do valor comprado por id de comprador,  e da quantidade de ordens por comprador onde eu incluí também o país de cada comprador (para ser usado para agrupar por país na querry principal).
* Subquerry 2 **country_or_other** 
> * A segunda sub querry é uma querry apenas de "organização" de dados onde eu criei uma nova coluna para classificar como "Other" os países com apenas 1 cliente. 
> * Para isso eu parti da primeira subquery e utilizei a função **CASE** para criar uma nova coluna "country modified" para adicionar uma nova coluna na subquery com "Other" nos países com apenas 1 cliente.
* Query principal
> * A query principal apresenta as estatísticas que queremos por país
> * Para isso ela agrupa os valores pela coluna country_modified da segunda sub querry. Foi preciso fazer um inner join entre as tabelas geradas pelas duas subqueries
> * Novamente eu usei a função **CASE** somente para **ordernar** os valores, forçando com que other ficasse na última posição: Para isso eu criei uma nova coluna *sort_values* e ordenei os valores por ela.

In [7]:
%%sql
with customer_agregate as 

(select 
    i.customer_id,
    c.country as country_name,
    round(sum(i.total),2) sales_by_customer,
    count(i.customer_id) orders_by_customer
    from customer c 
    inner join invoice i on c.customer_id = i.customer_id
    group by i.customer_id
    order by i.customer_id),

country_or_other as
 (select 
      ca.country_name as country_original,
      count(ca.customer_id) as users_by_country,
      case
        when count(ca.customer_id) = 1 then "Other" 
        else ca.country_name
        end as country_modified
      from customer_agregate ca
      group by ca.country_name)
       

select
    coo.country_modified as country,
    count(ca.customer_id) as number_of_customers,
    round(sum(ca.sales_by_customer),2) as total_sales,
    round(cast(sum(ca.sales_by_customer) as float)/cast(count(ca.customer_id) as float),2) as avg_sales_per_customer,
    round(cast(sum(ca.sales_by_customer) as float)/cast(sum(ca.orders_by_customer) as float),2) as avg_order_value,
    case
      when coo.country_modified = "Other" then 1
      else 0
      end as sort_column
    from customer_agregate ca
    inner join country_or_other coo on ca.country_name = coo.country_original
    group by coo.country_modified
    order by sort_column asc, 3 desc  

 * sqlite:///chinook.db
Done.


country,number_of_customers,total_sales,avg_sales_per_customer,avg_order_value,sort_column
USA,13,1040.49,80.04,7.94,0
Canada,8,535.59,66.95,7.05,0
Brazil,5,427.68,85.54,7.01,0
France,5,389.07,77.81,7.78,0
Germany,4,334.62,83.66,8.16,0
Czech Republic,2,273.24,136.62,9.11,0
United Kingdom,3,245.52,81.84,8.77,0
Portugal,2,185.13,92.57,6.38,0
India,2,183.15,91.58,8.72,0
Other,15,1094.94,73.0,7.45,1


* Vemos que os EUA são o primeiro país em valor comprado total, com praticamente do dobro do segundo colocado, que é o Canada.
* De um modo geral a medida que a quantidade de clientes por país aumenta, aumenta também o valor vendido total, já que o 'ticket médio' gasto em cada país é mais ou menos o mesmo, em torno de 80 dólares.
* A quantidade de usuários por país é bem pequena.

## 3.4. As pessoas estão comprando mais músicas avulsas ou os CDs inteiros?

* Sub Query 1 **tracks_per_album** 
> * Conta quantas faixas existem em cada álbum
* Sub Query 2 **track_per_invoice_per_album**
> * Para cada recibo, lista quais álbuns foram comprados, e quantas faixas desse álbum foram compradas
> * Para isso eu fiz um inner join entre as tabelas invoice line e track
> * Depois disso agrupei tanto por álbum quanto por recibo
* Sub Query 3 **albuns_per_invoice**
> * Com as duas queries que fizemos acima já temos quantas faixas existem por álbum (sub query 1) e quantas faixas de cada álbum foram compradas por recebo (sub query 2), agora vamos cruzar as duas informações
> * Para isso fiz um inner join entre as duas primeiras sub queries, utilizando como chave o número do álbum
> * Em seguida, usando a função CASE, inseri o valor 1 em uma nova coluna, sempre que a quantidade de faixas compradas para determinado álbum era igual a quantidade total de músicas no álbum (ainda temos várias linhas por invoice)
* Sub Query 4 **full_albuns_per_invoice**
> * Essa sub query é um agrupamento da sub query 3
> * Retorna apenas uma linha por invoice, e a informação de quantos álbuns existem no invoice, quantos álbuns completos existem naquele invoice
> * Quando um invoice é composto exclusivamente por cálbuns completos, retorna 1 na coluna 'album purchase'
* Query final
> * A query final é um agrupamento da sub query 4
> * Soma o total de invoices, e o total de invoices compostos exclusivamente por álbuns
> * Calcula a relação entre os 2

In [8]:
%%sql
with track_per_album as

(
select
    track.album_id,
    count(track.track_id) number_of_tracks_album
    from track
    group by 1
),

track_per_invoice_per_album as 

(
  select
  il.invoice_id,
  t.album_id,  
  count(t.track_id) tracks_bought
from invoice_line il
inner join track t on il.track_id = t.track_id
group by il.invoice_id,t.album_id),

albuns_per_invoice as

(select 
     tia.*,
     ta.number_of_tracks_album,
     case 
       when tia.tracks_bought = ta.number_of_tracks_album then 1
        else 0
       end as full_album_check       
    from track_per_invoice_per_album tia
    inner join track_per_album ta on tia.album_id = ta.album_id),

full_albuns_per_invoice as

(select
  invoice_id as invoice,
  count(album_id) as albuns_per_invoice,
  sum(full_album_check) as full_albuns_per_invoice,
  case 
    when count(album_id) = sum(full_album_check) then 1
      else 0
     end as album_purchase
  from albuns_per_invoice
  group by invoice_id)

select
   count(invoice) total_purchases,
   sum(album_purchase) album_purchases,
   Round(cast(sum(album_purchase) as float)/cast(count(invoice) as float),4) percentage_album_purchases,
   1 - Round(cast(sum(album_purchase) as float)/cast(count(invoice) as float),4) percentage_non_album_purchases
   from full_albuns_per_invoice

 * sqlite:///chinook.db
Done.


total_purchases,album_purchases,percentage_album_purchases,percentage_non_album_purchases
614,114,0.1857,0.8143


* Chegamos à conclusão de que as compras de músicas avulsas são extremamente comuns: 81,4% das compras contem pelo menos 1 música avulsa e apenas 18,6% são compras apenas de álbuns inteiros. 